<a href="https://colab.research.google.com/github/JunePark16/JunePark16/blob/main/hw3_s3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Setup

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [5]:
"""
Change directory to where this file is located
"""
%cd /content/drive/MyDrive/MLVU_HW3/MLVU_HW3/S3D

/content/drive/MyDrive/MLVU_HW3/MLVU_HW3/S3D


In [3]:
"""
Unzip the UCF11 zip file.
- Run this cell only once.
"""
# !unzip -n ./UCF11.zip -d .

'\nUnzip the UCF11 zip file.\n- Run this cell only once.\n'

# Import Modules

In [6]:
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from glob import glob
from tqdm import tqdm
from IPython.display import HTML
from sklearn.model_selection import train_test_split

In [7]:
"""
import more modules you need
"""
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten, Dense, Conv3D, ReLU, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2

In [8]:
"""
Make sure your runtime type is GPU!
"""
cpus = tf.config.list_physical_devices('CPU')
gpus = tf.config.list_physical_devices('GPU')
print('Num_CPUs:{}, List:{}'.format(len(cpus), cpus))
print('Num_GPUs:{}, List:{}'.format(len(gpus), gpus))

Num_CPUs:1, List:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Num_GPUs:1, List:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Utils

In [9]:
def bytes_feature(value):
    """
    Returns a bytes_list from a string / byte.

    Do NOT modify this method.
    """
    if not isinstance(value, list):
        value = [value]
        # bytes_feature 의 인자에는 list 가 들어가도록 변환
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def int64_feature(value):
    """
    Returns a float_list from a float / double.

    Do NOT modify this method.
    """
    if not isinstance(value, list):
        value = [value]
        # int64_feature 의 인자에는 list 가 들어가도록 
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def float_feature(value):
    """
    Returns a float_list from a float / double.

    Do NOT modify this method.
    """
    if not isinstance(value, list):
        value = [value]
        # float_feature 의 인자에는 list 가 들어가도록
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
    
def plot_frames(videos, labels, grid_width, grid_height, figure_width=15, figure_height=5, y_hats=None):
    """
    Plots videos and labels.

    Do NOT modify this method.
    """
    fig, ax = plt.subplots(grid_height, grid_width)
    fig.set_size_inches(figure_width, figure_height)
    video_idx = 0
    imgs = []
    for i in range(0, grid_height):
        for j in range(0, grid_width):
            video = videos[video_idx]
            label = labels[video_idx]
            title_color = 'k'
            if y_hats is None:
                label_idx = int(label)
            else:
                label_idx = int(y_hats[video_idx])
                if int(labels[video_idx]) != label_idx:
                    title_color = 'r'
            label = InputPipelineBuilder.LABELS[label_idx]
            ax[i][j].axis('off')
            ax[i][j].set_title(label, color=title_color)
            
            for frame_idx, frame in enumerate(video):
                img = ax[i][j].imshow(frame, animated=True, aspect='auto')
                if video_idx==0:
                    imgs.append([img])
                else:
                    imgs[frame_idx].extend([img])
            ani = animation.ArtistAnimation(fig, imgs, interval=150, blit=False)
            video_idx += 1
    plt.close(fig)
    os.makedirs('./temp', exist_ok=True)
    ani.save(f'./temp/frames.gif', writer='pillow', fps=10)
    display(HTML(ani.to_html5_video()))

def plot_history(history):
    """
    Plots training history.
    
    Do NOT modify this method.
    """
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

# UCF11

In [10]:
class InputPipelineBuilder:
    """
    UCF11 Input Pipeline
    - 1. Contains configuration and information about the inputs.
    - 2. Splits the dataset into train/valid/test subets.
    - 3. Load videos and sample frames to use, followed by preprocessing.
    - 4. Store the sampled frames as TFRecord files.
    - 5. Returns TF dataset objects built directly from the TFRecord files.
    """
    # pipeline configurations 
    FRAME_SIZE = (56,56)      
    NUM_FRAMES_PER_VIDEO = 16   # video 당 16개의 프레임을 보겠다
    NUM_VIDEOS_PER_RECORD = 64  
    SUBSETS = ['train', 'valid', 'test']
    LABELS = ['basketball', 'biking', 'diving', 'golf_swing',
              'horse_riding', 'soccer_juggling', 'swing', 'tennis_swing',
              'trampoline_jumping', 'volleyball_spiking', 'walking']
    NUM_LABELS = 11
    INPUT_SHAPE = [NUM_FRAMES_PER_VIDEO, *FRAME_SIZE, 3]    # *FRAME_SIZE 는 tuple 로 component 를 받는다.
    
    def __init__(self, dataset_dir, tfrecord_dir):
        """
        Loads and splits UCF11 to train, validation, and test sets.

        Do NOT modify this method.
        """
        self.dataset_dir = dataset_dir
        self.tfrecord_dir = tfrecord_dir
        self.x = None   # self.x 에는 train/validation/test set 의 data
        self.y = None   # self.y 에는 train/validataion/test set 의 label
        
    def __str__(self):
        """
        Print the input pipeline configuration and data subsets.
        
        Do NOT modify this method.
        """
        info = ''
        info += f'Labels: {InputPipelineBuilder.LABELS}\n'
        info += f'Dimension to extract per video: {[InputPipelineBuilder.NUM_FRAMES_PER_VIDEO, *InputPipelineBuilder.FRAME_SIZE, 3]}\n'
        info += 'Dataset splitted: '
        if self.x:
            info += '/'.join(InputPipelineBuilder.SUBSETS)
            info += ' [' + '|'.join([str(len(self.x[subset])) for subset in InputPipelineBuilder.SUBSETS]) + ']'
        return info
    
    def split_dataset(self, valid_size=0.2, test_size=0.1, random_state=None):
        """
        Read video lists and split them into subsets
        
        Do NOT modify this method.
        """
        videos = []
        labels = []
        for label_int, label_str in enumerate(InputPipelineBuilder.LABELS):
            # InputPipelineBuilder.LABELS 에는 11개의 label
            path = os.path.join(self.dataset_dir, label_str)
            temp = glob(path+'/**/*.mpg')
            videos.extend(temp)    # extend 는 iterable element 를 넣는다.
            labels.extend([label_int,] * len(temp))
            # if label_int == 0 :
            #     print(f'len(temp) = {len(temp)}')
        x_train, x_test, y_train, y_test   = train_test_split(videos, labels, test_size=test_size, stratify=labels,
                                                              shuffle=True, random_state=random_state)
        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=valid_size, stratify=y_train,
                                                              shuffle=True, random_state=random_state)
        self.x = {'train':x_train, 'valid':x_valid, 'test':x_test}  # self.x 는 video 를 담은 딕셔너리
        self.y = {'train':y_train, 'valid':y_valid, 'test':y_test}  # selfl.y 는 label 을 담은 딕셔너리
        
    def convert_dataset_to_tfrecord(self, subset='train'):
        """
        Load and store videos in TFRecord format.
        
        Do NOT modify this method.
        """
        assert subset in InputPipelineBuilder.SUBSETS, 'subset supported: {}'.format(InputPipelineBuilder.SUBSETS)
        # InputPipilineBuilder.SUBSETS 는 train, valid, test 중 하나
        videos = self.x[subset]
        labels = self.y[subset]
        tfrecord_dir = os.path.join(self.tfrecord_dir, subset) # train, valid, test
        files = glob(tfrecord_dir+'/*')
        for file in files:
            os.remove(file)
            # 우선 tfrecord_dir + subset 폴더 초기화
        os.makedirs(tfrecord_dir, exist_ok=True)
            # tfrecord_dir + subset 폴더가 존재해도 오류 발생시키지 X
        for i in tqdm(range(0, len(videos), InputPipelineBuilder.NUM_VIDEOS_PER_RECORD)):   
            # InputPipelineBuilder.NUM_VIDEOS_PER_RECORD : 64 -> record(파일) 하나 당 64개의 비디오를 저장
            # tqdm 은 process 진행 바     
            tfrecords_file_path = os.path.join(tfrecord_dir, 'tfr_'+str(i))
            with tf.io.TFRecordWriter(tfrecords_file_path) as record_writer:
                upto = min(i+InputPipelineBuilder.NUM_VIDEOS_PER_RECORD, len(videos))
                for video, label in zip(videos[i:upto], labels[i:upto]):
                        example = self.encode_tfrecord(video, label)    ## example 은 byte sequence
                        record_writer.write(example.SerializeToString())    # record 파일을 2진 문자열로 직렬화
        print("TFRecord Conversion DONE\n")
        
    def convert_tfrecord_to_tfdataset(self, subset='train'):
        """
        Creates a tf.dataset from TFRecord files.

        Do NOT modify this method.
        """
        assert subset in InputPipelineBuilder.SUBSETS, 'subset supported: {}'.format(InputPipelineBuilder.SUBSETS)
        # InputPipelineBuilder.SUBSETS : train, valid, test 중 하나
        tfrecord_files = glob(os.path.join(self.tfrecord_dir, subset + '/*'))
        dataset = tf.data.TFRecordDataset(tfrecord_files).map(lambda x: self.decode_tfrecord(x))    # 2진 문자열을 video 로 변환
        return dataset  
        
    def load_video(self, video):
        """
        Loads a video from a given path and samples a certain number of frames at equal distance.
        Preprocsses each frame while sampling.

        Question (a)
        - Use cv2.VideoCapture to access the contents of the video at the given path.
        - Computes the frame indices to sample using ...
        - the number of the total frames achieved with the 'get' method of cv2.VideoCapture.
        - the number of frames to sample is stored as a class variable: InputPipelineBuilder.NUM_FRAMES_PER_VIDEO
        - Read each frame at these indices and apply self.preprocess 
        - Release the capture object at the end.

        - Refer to OpenCV VideoCapture Documentation for further information!

        Inputs
        - video: a file path to a video.
        Returns
        - frames: a numpy array of frames extracted from the video.
        """
        frames=[]#

        cap = cv2.VideoCapture(video)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        dist = frame_count/InputPipelineBuilder.NUM_FRAMES_PER_VIDEO
        for i in range(frame_count) :
            # if i % int(cap.get(cv2.CAP_PROP_FRAME_COUNT))/InputPipelineBuilder.NUM_FRAMES_PER_VIDEO == 0 :
            if i % dist == 0 :
                retval, frame = cap.read()
                frame = self.preprocess(frame)    ## ?? frame = preprocess(frame)
                frames.append(frame)
        cap.release()
        # cv2.destroyAllwindows() 
        return np.asarray(frames) #
    
    def preprocess(self, frame):
        """
        Preprocess the given frame.

        Question (a)
        - Use OpenCV(cv2) methods.
        - 1. Convert the color format from BGR to RGB; OpenCV reads frames in the BGR format.
        - 2. Resize the frame to InputPipelineBuilder.FRAME_SIZE.
        - 3. Normalize.
        
        Inputs
        - frame
        Returns
        - frame: a frame preprocssed.
        """
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 1.
        frame = cv2.resize(frame, InputPipelineBuilder.FRAME_SIZE)  # 2.
        frame = cv2.normalize(frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX) ## beta = 255? , dtype = cv2.CV_32F?
        return frame
        
    def encode_tfrecord(self, video, label):
        # video 랑 label 을 tf.train.Example 로 변환 -> byte sequence
        # output 은 byte sequence 로 이루어저 잇고 이것들로 TFRecord file 을 만듬

        # encode 는 기존 데이터를 byte 로 바꾼다는 뜻

        # encoding 할 video 은 np.array (->bytes)
        # encoding 할 label 은 int64 type
        """
        Convert a set of video and label as tf.train.Example.
        An example contains a byte sequence of one data point and these examples comprise a TFRecord file.

        Question (b)
        - Load frames from a video and use methods in Utils section in order to ...
        - Convert frames to byte_features. You will need to convert np.array frames to a byte array beforehand.
        - Convert label to int64_features.

        - Use these features to initialize tf.train.Features and tf.train.Example
        - Refer to TensorFlow TFRecord Tutorial for further information!

        Inputs
        - video: a video path
        - label: an action label
        Returns
        - example: tf.train.Example
        """
        frames = self.load_video(video) # frames : nparray
        feature = {
            'frames' : bytes_feature(frames.tobytes()),
            'label' : int64_feature(label)
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))  ## ??
        return example  # example 은 byte sequence

    def decode_tfrecord(self, tfrecord):
        # decode 는 byte 를 기존 데이터 포맷으로 바꾼다.
        """
        Decode a tfrecord item.
        You need to specify how many bytes to read from this byte sequence per feature to retrieve a data point.

        Question (b)
        - Use tf.io.FixedLenFeature([], 'type of feature')
        - Use tf.io.parse_single_example, tf.io.decode_raw
            - Use InputPipelineBuilder.INPUT_SHAPE to reshape after decoding frames.
            - Use tf.cast to cast label to tf.int32 after decoding label.    
                ## tf.int32???

        - Refer to TensorFlow TFRecord Tutorial for further information!

        Inputs
        - tfrecord
        Returns
        - frames
        - labels
        """
        feature_desc = {
            'frames' : tf.io.FixedLenFeature([], tf.string ),
            'label' : tf.io.FixedLenFeature([], tf.int64 )
        }
        # need to specify how many bytes to read from this byte sequence per feature to retrieve a data point.

        ## label : tf.string -> tf.int64
        ## label 을 tf.string(:bytes) 으로 변환하면 안될 것 같은데..
        
        frames = tf.io.parse_single_example(tfrecord , feature_desc)['frames']
        frames = tf.io.decode_raw(frames, tf.float32) ## tf.float32 -> tf.int32 -> tf.float32
        frames = tf.reshape(frames, InputPipelineBuilder.INPUT_SHAPE)

        label = tf.io.parse_single_example(tfrecord, feature_desc)['label']
        label = tf.io.decode_raw(label, tf.float32)
        label=tf.cast(label, tf.int32)  
        # Use tf.cast to cast label to tf.int32 after decoding label.
        return frames, label

In [11]:
"""
Builds a UCF11 input pipeline
"""
input_pipeline_builder = InputPipelineBuilder(dataset_dir='./UCF11', tfrecord_dir='./tfrecord')
input_pipeline_builder.split_dataset(valid_size=0.2, test_size=0.1)
print(input_pipeline_builder)

Labels: ['basketball', 'biking', 'diving', 'golf_swing', 'horse_riding', 'soccer_juggling', 'swing', 'tennis_swing', 'trampoline_jumping', 'volleyball_spiking', 'walking']
Dimension to extract per video: [16, 56, 56, 3]
Dataset splitted: train/valid/test [1149|288|160]


In [ ]:
"""
Convert videos into TFRecord files
You don't need to run this cell more than once unless TFRecord files need updates.
"""
##### 딱 한 번만 실행 #####
use_gpu = True #False
cpus = tf.config.list_logical_devices('CPU')
gpus = tf.config.list_logical_devices('GPU')
with tf.device(gpus[0].name if use_gpu else cpus[0].name):
    input_pipeline_builder.convert_dataset_to_tfrecord(subset='train')
    input_pipeline_builder.convert_dataset_to_tfrecord(subset='valid')
    input_pipeline_builder.convert_dataset_to_tfrecord(subset='test')

  0%|          | 0/18 [00:00<?, ?it/s]

In [118]:
train_ds_preprocessed = input_pipeline_builder.convert_tfrecord_to_tfdataset(subset='train')
valid_ds_preprocessed = input_pipeline_builder.convert_tfrecord_to_tfdataset(subset='valid')
test_ds_preprocessed = input_pipeline_builder.convert_tfrecord_to_tfdataset(subset='test')
videos, labels = next(iter(train_ds_preprocessed.shuffle(100).batch(16).take(1)))
plot_frames(videos, labels, 8, 2)

InvalidArgumentError: ignored

# S3D

In [ ]:
def sep_conv3d_bn(input_tensor, kernel_size, filters, strides=1, padding='same', separate=False):
    """
    Builds a separable Conv3D+BN layer.
    
    Question (c)
    - Use ReLU to each Conv3D+BN for activation
    - Use l2 regularizer for each Conv3D.
    - for separated Conv3Ds, apply the same number of filters for both.
    - when separate=False, this is just a normal Conv3D.

    Inputs
    - input_tensor
    - kernel_size: an integer to indicate kernel_size  
    - filters: an integer to indicate the number of filters    -> integer && list 가능
    - strides: an integer to indicate strides
    Returns
    - output_tensor
    """
    if separate:        
        # Spatial Conv : 1x3x3
        output_tensor = tf.keras.layers.Conv3D(
            filters = filters[0],
            kernel_size = (1, kernel_size, kernel_size),
            activation = 'relu',
            kernel_regularizer = 'l2')(input_tensor)
        # Temporal Conv : 3x1x1
        output_tensor = tf.keras.layers.Conv3D(
            filters = filters[1],
            kernel_size = (kernel_size, 1, 1),
            activation = 'relu',
            kernel_regularizer = 'l2')(output_tensor)
        
    else:
        # Spatio-temporal Conv
        output_tensor = tf.keras.layers.Conv3D(
            filters = filters,
            kernel_size = (kernel_size, kernel_size, kernel_size),
            activation = 'relu',
            kernel_regularizer = 'l2')(input_tensor)
    
    # BN
    output_tensor = tf.keras.layers.BatchNormalization()(output_tensor)    ## ??
    return output_tensor

def sep_inception(input_tensor, filters1, filters2, filters3, filters4):
    """
    Builds a separable Inception block.
    
    Question (c)
    - Refer to the figure 2 for the overall architecture.
    - Use sep_conv3d_bn above.

    Inputs
    - input_tensor
    - filters1: an integer to indicate the number of filters
    - filters2: a list of integers to indicate the number of filters
    - filters3: a list of integers to indicate the number of filters
    - filters4: an integer to indicate the number of filters
    Returns
    - output_tensor
    """
    # 1x1x1 Conv
    conv1 = sep_conv3d_bn(input_tensor = input_tensor,
                          kernel_size = 1,
                          filters = filters1)
    # 1x1x1 Conv + 3x3x3 Conv Separated
    conv2 = sep_conv3d_bn(input_tensor = input_tensor,
                          kernel_size = 1,
                          filters = filters2[0])
    conv2 = sep_conv3d_bn(input_tensor = conv2,
                          kernel_size = 3,
                          filters = filters2[1],
                          separate = True)
    # 1x1x1 Conv + 3x3x3 Conv Separated
    conv3 = sep_conv3d_bn(input_tensor = input_tensor,
                          kernel_size = 1,
                          filters = filters3[0])
    conv3 = sep_conv3d_bn(input_tensor = conv2,
                          kernel_size = 3,
                          filters = filters3[1],
                          separate = True)
    # 3x3x3 MaxPool + 1x1x1 Conv
    conv4 = tf.keras.layers.MaxPooling3D(pool_size=(3,3,3))(input_tensor)
    conv4 = sep_conv3d_bn(input_tensor = conv4,
                          kernel_size = 3,
                          filters = filters4)
    # Concatenation
    output_tensor = tf.keras.layers.Concatenate(axis=0)([conv1, conv2, conv3, conv4])
    return output_tensor
    
def S3D(input_shape):
    """
    Builds a S3D model.

    Question (d)
    - Refer to the figure 2 for the overall architecture.
    - You can try different layers for fc layers!

    Inputs
    - input_shape: the shape of the input image.
    Returns
    - a TF Keras Model model of S3D
    """
    # input layer
    input_tensor = Input(shape=input_shape)

    # stem
    x = sep_conv3d_bn(input_tensor, kernel_size = 3, filters = 192, strides = 2)
    x = tf.keras.layers.MaxPooling3D(pool_size=(1,2,2))(x)
    # separable inception block 1 + max pooling
    x = sep_inception(input_tensor = x, 
                      filter1 = 64,
                      filter2 = [96,128],
                      filter3 = [16,32],
                      filter4 = 32)
    x = tf.keras.layers.MaxPooling3D(pool_size=(2,2,2))(x)
    # separable inception block 2 + max pooling
    x = sep_inception(input_tensor = x, 
                      filter1 = 64,
                      filter2 = [96,128],
                      filter3 = [16,32],
                      filter4 = 32)
    x = tf.keras.layers.MaxPooling3D(pool_size=(2,2,2))(x)
    # 1x1x1 Conv
    x = sep_conv3d_bn(x, kernel_size = 1, filters = 256)

    ### fc layers -- tf.keras.layers 추가
    x = tf.keras.layers.Flatten()(x)    
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, kernel_regularizer=l2(0.001))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(InputPipelineBuilder.NUM_LABELS, kernel_regularizer=l2(0.001), activation='softmax')(x)
    
    return tf.keras.Model(input_tensor, x)

In [ ]:
### 
# Question (e)
# Train your S3D to achieve test accuracy 80% and above.
# You can try or add other training options such as SGD or callbacks to schedule learning rates if you want.
### 

In [ ]:
"""
Training Args
- modify as you want
- set load_weights=True to load the previous best checkpoint.
"""
batch_size = 16
num_epochs = 15
learning_rate = 0.0005
load_weights = False
print_summary = False
checkpoint_path = './ckpts/s3d_ckpt'

In [ ]:
"""
Utilize TF dataset methods to expedite I/O.
"""
train_ds_preprocessed = input_pipeline_builder.convert_tfrecord_to_tfdataset(subset='train')
valid_ds_preprocessed = input_pipeline_builder.convert_tfrecord_to_tfdataset(subset='valid')
test_ds_preprocessed = input_pipeline_builder.convert_tfrecord_to_tfdataset(subset='test')

train_ds, train_steps = train_ds_preprocessed.repeat().shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE),\
                        len(input_pipeline_builder.x['train'])//batch_size
valid_ds, valid_steps = valid_ds_preprocessed.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE),\
                        len(input_pipeline_builder.x['valid'])//batch_size
test_ds, test_steps = test_ds_preprocessed.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE),\
                        len(input_pipeline_builder.x['test'])//batch_size

In [ ]:
tf.keras.backend.clear_session()

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy()

s3d = S3D(InputPipelineBuilder.INPUT_SHAPE)
s3d.compile(optimizer, loss, metrics=metric)
if load_weights:
    s3d.load_weights(checkpoint_path)
if print_summary:
    s3d.summary()

callbacks = []
callbacks.append(tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True))

In [ ]:
history = s3d.fit(train_ds,
                  steps_per_epoch=train_steps,
                  validation_data=valid_ds,
                  validation_steps=valid_steps,
                  callbacks=callbacks,
                  epochs=num_epochs,
                  verbose=1)
plot_history(history)

In [ ]:
"""
Measure test accuracy
"""
s3d.evaluate(test_ds, steps=test_steps)
videos, labels = next(iter(test_ds.shuffle(buffer_size=200).take(1)))
y_hats = np.argmax(s3d.predict(videos), axis=1)
plot_frames(videos, labels, 8, 2, y_hats=y_hats)